In [0]:
import json
import sklearn
import os
import nltk
import spacy
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from nltk.tokenize import RegexpTokenizer, wordpunct_tokenize, sent_tokenize
from spacy.tokenizer import Tokenizer
from spacy.pipeline import Sentencizer
import operator
from spacy.lang.en import English
nlp = English()

import readability

from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.preprocessing import Normalizer,power_transform
from sklearn import svm
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
with open("summary_quality/train_data.json",'r') as fin:
    train_data = json.load(fin)

In [0]:
with open("summary_quality/test_data.json",'r') as fin:
    test_data = json.load(fin)

In [0]:
Summary_list = []

In [0]:
#files= os.listdir('summary_quality/summaries') #get all file names under the training dataset

for file in list(train_data): 
    f = open('summary_quality/summaries/'+file, mode = 'r',encoding='latin-1')
    article = f.read()
    #all_text.append(article.lower()) # lowercase all the words
    f.close()
    Summary_list.append([file, article])

In [0]:
test_summary = []
for file in list(test_data): 
    f = open('summary_quality/summaries/'+file, mode = 'r',encoding='latin-1')
    article = f.read()
    #all_text.append(article.lower()) # lowercase all the words
    f.close()
    test_summary.append([file, article])

#### 4.1.1

In [0]:
nlp = spacy.load("en_core_web_sm")

In [0]:
sentencizer = nlp.create_pipe("sentencizer")
nlp.add_pipe(sentencizer)

In [0]:
def token_it(text):
    doc = nlp(text)
    unigram = []
    bigram = []
    for i in range(len(doc)):
        unigram.append(doc[i].text)
        if i>=1:
            bigram.append([doc[i-1].text, doc[i].text])
    return unigram, bigram

In [0]:
def sentence_token(text):
    
    doc = nlp(text)
    return list(doc.sents)

In [0]:
def count_repetitive(gram_List):
    repetitive_num = 0
    for j in range(len(gram_List)):
        if j >= 1:
            if str(gram_List[j]) == str(gram_List[j-1]):
                #print(str(gram_List[j]),str(gram_List[j-1]))
                repetitive_num +=1
            
    return repetitive_num

In [0]:
def readability_score(all_sentences):
    read_score = []
    for i in all_sentences:

        if any(c.isalpha() for c in str(i)) is True:
            
            text = (str(i))
        
            results = readability.getmeasures(text, lang='en')
            read_score.append(results['readability grades']['FleschReadingEase'])
    if len(read_score)>0:
        return read_score
    else:
        return [0]


In [0]:
def summary_vector(data_dic, filename):
    temp_dic = data_dic[filename]
    sent_vec = [int(temp_dic['coherence']),int(temp_dic['grammaticality']),int(temp_dic['nonredundancy'])]
    
    return sent_vec

In [0]:
Grammaticality_Score_Vec = []
Grammaticality_Score_Y = []
for i in Summary_list:
    unigram, bigrm = token_it(i[1].lower().replace('\n',' '))
    
    sentences = sentence_token(i[1])
    temp_vector = [count_repetitive(unigram), count_repetitive(list(bigrm)),
                   min(readability_score(sentences))]
    Grammaticality_Score_Y.append(summary_vector(train_data, i[0])[1])
    Grammaticality_Score_Vec.append(temp_vector)

In [0]:
Grammaticality_test_Vec = []
Grammaticality_test_Y = []
for i in test_summary:

    unigram, bigrm = token_it(i[1].lower().replace('\n',' '))
    
    sentences = sentence_token(i[1])
    temp_vector = [count_repetitive(unigram), count_repetitive(list(bigrm)),
                   min(readability_score(sentences))]
    Grammaticality_test_Y.append(summary_vector(test_data, i[0])[1])
    Grammaticality_test_Vec.append(temp_vector)

In [0]:
clf_gramma = svm.SVC(gamma='auto_deprecated')
clf_gramma.fit(Grammaticality_Score_Vec, Grammaticality_Score_Y)  

/Users/xizheng/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
predict_gramma_1 = clf_gramma.predict(Grammaticality_test_Vec)

In [0]:
gramma_MSE_1 = mean_squared_error(Grammaticality_test_Y,predict_gramma_1)

In [0]:
gramma_pearsonr_1 = pearsonr(Grammaticality_test_Y,predict_gramma_1)

In [0]:
print('The MSE is '+str(gramma_MSE_1) +'. The pearson is '+str(gramma_pearsonr_1) +'.')

The MSE is 0.7564766839378239. The pearson is (0.20775594900478267, 0.0037412836241748322).


#### 4.1.2

In [0]:
def all_repeat_uni(texts):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(texts)
    uni = {}
    for i in range(len(doc)):
        if doc[i].text in uni:
            uni[doc[i].text] += 1
        if doc[i].text not in uni:
            uni[doc[i].text] = 1
    
    repeat_grams = 0
    for i in list(uni.keys()):
        if uni[i] >1:
            repeat_grams +=1
    return repeat_grams

In [0]:
def countgram(gram_List):
    count_dic = {}
    
    for j in gram_List:
        if str(j) in count_dic:
            count_dic[str(j)] += 1
        if str(j) not in count_dic:
            count_dic[str(j)] = 1
    return count_dic

In [0]:
Grammaticality_Score_Vec_1 = []

for i in range(len(Grammaticality_Score_Vec)):
    temp_text = Summary_list[i][1]
    unigram, bigrm = token_it(temp_text.lower().replace('\n',' '))
    Grammaticality_Score_Vec_1.append(Grammaticality_Score_Vec[i]+
                                      [len(countgram(unigram))/len(sentence_token(temp_text))])

In [0]:
Grammaticality_test_Vec_1 = []

for i in range(len(Grammaticality_test_Vec)):
    temp_text = test_summary[i][1]
    unigram, bigrm = token_it(temp_text.lower().replace('\n',' '))
    Grammaticality_test_Vec_1.append(Grammaticality_test_Vec[i]+
                                      [len(countgram(unigram))/len(sentence_token(temp_text))])

In [0]:
def normalization(X):
    return power_transform(X, method='yeo-johnson')

In [0]:
Grammaticality_Score_Vec_1_norm = normalization(Grammaticality_Score_Vec_1)

In [0]:
Grammaticality_test_Vec_1_norm = normalization(Grammaticality_test_Vec_1)

In [0]:
clf_gramma = svm.SVC(gamma='auto_deprecated')
clf_gramma.fit(Grammaticality_Score_Vec_1_norm, Grammaticality_Score_Y)  

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
predict_gramma_2 = clf_gramma.predict(Grammaticality_test_Vec_1_norm)

In [0]:
gramma_MSE_2 = mean_squared_error(Grammaticality_test_Y,predict_gramma_2)

In [0]:
gramma_pearsonr_2 = pearsonr(Grammaticality_test_Y,predict_gramma_2)

In [0]:
print('The MSE is '+str(gramma_MSE_2) +'. The pearson is '+str(gramma_pearsonr_2) +'.')

The MSE is 0.7150259067357513. The pearson is (0.2821574928897053, 7.023937045149804e-05).


In [0]:
Grammaticality_Score_Vec_2 = []

for i in range(len(Grammaticality_Score_Vec)):
    temp_text = Summary_list[i][1]
    unigram, bigrm = token_it(temp_text.lower().replace('\n',' '))
    Grammaticality_Score_Vec_2.append(Grammaticality_Score_Vec[i]+
                                      [Grammaticality_Score_Vec[i][0]/len(unigram)])

In [0]:
Grammaticality_test_Vec_2 = []

for i in range(len(Grammaticality_test_Vec)):
    temp_text = test_summary[i][1]
    Grammaticality_test_Vec_2.append(Grammaticality_test_Vec[i]+
                                      [Grammaticality_test_Vec[i][0]/len(unigram)])

In [0]:
clf_gramma = svm.SVC(gamma='auto_deprecated')
clf_gramma.fit(Grammaticality_Score_Vec_2, Grammaticality_Score_Y)  

/Users/xizheng/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
predict_gramma_3 = clf_gramma.predict(Grammaticality_test_Vec_2)

In [0]:
gramma_MSE_3 = mean_squared_error(Grammaticality_test_Y,predict_gramma_3)

In [0]:
gramma_pearsonr_3 = pearsonr(Grammaticality_test_Y,predict_gramma_3)

In [0]:
print('The MSE is '+str(gramma_MSE_3) +'. The pearson is '+str(gramma_pearsonr_3) +'.')

The MSE is 0.7564766839378239. The pearson is (0.20775594900478267, 0.0037412836241748322).


#### 4.2.1

In [0]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [0]:
from gensim.models import KeyedVectors

In [0]:
google_vecs = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [0]:
def remove_stop(grams):
    new_grams = []
    for i in grams:
        if str(i) not in spacy_stopwords:
            new_grams.append(i)
    return new_grams

In [0]:
def compute_similarity(sent_list):
    sentence_embedding = []
    for i in sent_list:
        words_embedding = []
        for words in str(i).split(' '):
            if str(words).lower() in google_vecs:
                words_embedding.append(google_vecs[str(words).lower()])
            else:
                words_embedding.append(np.zeros(google_vecs.vector_size))
        sentence_embedding.append(np.mean(words_embedding,axis=0))
    return sentence_embedding

In [0]:
def generate_similarity(embedding):
    cosSim = []
    for i in embedding:
        for j in embedding:
            if str(i) !=str(j):
                cosSim.append(cosine_similarity([i], [j])[0][0])
    if len(cosSim) == 0:
        max_sim = 0
    else:
        max_sim = sorted(cosSim, reverse=True)[0]
    return max_sim

In [0]:
Redundancy_Score_Vec = []
Redundancy_Score_Y = []
for i in Summary_list:
    unigram, bigrm = token_it(i[1].lower().replace('\n',' '))
    unigram_dic = dict(sorted(countgram(remove_stop(unigram)).items(), key=operator.itemgetter(1),reverse=True))
    bigram_dic = dict(sorted(countgram(bigrm).items(), key=operator.itemgetter(1),reverse=True))
    sentences = sentence_token(i[1])
    temp_vector = [unigram_dic[list(unigram_dic)[0]], bigram_dic[list(bigram_dic)[0]],generate_similarity(compute_similarity(sentences))]

    Redundancy_Score_Vec.append(temp_vector)
    Redundancy_Score_Y.append(summary_vector(train_data, i[0])[2])

In [0]:
Redundancy_Score_Vec_norm = normalization(Redundancy_Score_Vec)

In [0]:
Redundancy_test_Vec = []
Redundancy_test_Y = []
for i in test_summary:
    unigram, bigrm = token_it(i[1].lower().replace('\n',' '))
    unigram_dic = dict(sorted(countgram(remove_stop(unigram)).items(), key=operator.itemgetter(1),reverse=True))
    bigram_dic = dict(sorted(countgram(bigrm).items(), key=operator.itemgetter(1),reverse=True))
    sentences = sentence_token(i[1])
    temp_vector = [unigram_dic[list(unigram_dic)[0]], bigram_dic[list(bigram_dic)[0]],generate_similarity(compute_similarity(sentences))]

    Redundancy_test_Vec.append(temp_vector)
    Redundancy_test_Y.append(summary_vector(test_data, i[0])[2])

In [0]:
Redundancy_test_Vec_norm = normalization(Redundancy_test_Vec)

In [0]:
clf_red = svm.SVC(gamma='auto_deprecated')
clf_red.fit(Redundancy_Score_Vec_norm, Redundancy_Score_Y)  

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
predict_red_1 = clf_red.predict(Redundancy_test_Vec_norm)
red_MSE_1 = mean_squared_error(Redundancy_test_Y,predict_red_1)
red_pearsonr_1 = pearsonr(Redundancy_test_Y,predict_red_1)
print('The MSE is '+str(red_MSE_1) +'. The pearson is '+str(red_pearsonr_1) +'.')

The MSE is 0.8031088082901554. The pearson is (0.11481526234107749, 0.111842589457841).


#### 4.2.2

#### Find each token's pos, calculate the frequencies of all the pos associate with the tokens, and use the maximum value as a new feature value.

In [0]:
def find_pos(text):
    pos_dic = {}
    
    nlp = spacy.load("en_core_web_sm")
    for token in nlp(text):
        ps = token.pos_
        if str(ps) in pos_dic:
            pos_dic[str(ps)] +=1
        if str(ps) not in pos_dic:
            pos_dic[str(ps)] =1
    return pos_dic

In [0]:
Redundancy_Score_Vec_1 = []

for i in range(len(Redundancy_Score_Vec)):
    temp_text = Summary_list[i][1]
    unigram_pos = dict(sorted(find_pos(temp_text).items(), key=operator.itemgetter(1),reverse=True))
    Redundancy_Score_Vec_1.append(Redundancy_Score_Vec[i]+
                                      [unigram_pos[list(unigram_pos)[0]]])

In [0]:
Redundancy_test_Vec_1 = []

for i in range(len(Redundancy_test_Vec)):
    temp_text = test_summary[i][1]
    unigram_pos = dict(sorted(find_pos(temp_text).items(), key=operator.itemgetter(1),reverse=True))
    Redundancy_test_Vec_1.append(Redundancy_test_Vec[i]+
                                      [unigram_pos[list(unigram_pos)[0]]])

In [0]:
Redundancy_Score_Vec_1_norm = normalization(Redundancy_Score_Vec_1)

In [0]:
Redundancy_test_Vec_1_norm = normalization(Redundancy_test_Vec_1)

In [0]:
clf_red = svm.SVC(gamma='auto_deprecated')
clf_red.fit(Redundancy_Score_Vec_1_norm, Redundancy_Score_Y)  

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
predict_red_2 = clf_red.predict(Redundancy_test_Vec_1_norm)
red_MSE_2 = mean_squared_error(Redundancy_test_Y,predict_red_2)
red_pearsonr_2 = pearsonr(Redundancy_test_Y,predict_red_2)
print('The MSE is '+str(red_MSE_2) +'. The pearson is '+str(red_pearsonr_2) +'.')

The MSE is 0.7979274611398963. The pearson is (0.13338004311664917, 0.06442765910550147).


#### Find each token's tag, calculate the frequencies of all the tag associate with the tokens, and use the maximum value as a new feature value.

In [0]:
def find_tag(text):
    tg_dic = {}
    
    nlp = spacy.load("en_core_web_sm")
    for token in nlp(text):
        tg = token.tag_
        if str(tg) in tg_dic:
            tg_dic[str(tg)] +=1
        if str(tg) not in tg_dic:
            tg_dic[str(tg)] =1
    return tg_dic

In [0]:
Redundancy_Score_Vec_2 = []

for i in range(len(Redundancy_Score_Vec)):
    temp_text = Summary_list[i][1]
    unigram_tag= dict(sorted(find_tag(temp_text).items(), key=operator.itemgetter(1),reverse=True))
    Redundancy_Score_Vec_2.append(Redundancy_Score_Vec[i]+
                                      [unigram_tag[list(unigram_tag)[0]]])

In [0]:
Redundancy_test_Vec_2 = []

for i in range(len(Redundancy_test_Vec)):
    temp_text = test_summary[i][1]
    unigram_tag = dict(sorted(find_tag(temp_text).items(), key=operator.itemgetter(1),reverse=True))
    Redundancy_test_Vec_2.append(Redundancy_test_Vec[i]+
                                      [unigram_tag[list(unigram_tag)[0]]])

In [0]:
Redundancy_Score_Vec_2_norm = normalization(Redundancy_Score_Vec_2)

In [0]:
Redundancy_test_Vec_2_norm = normalization(Redundancy_test_Vec_2)

In [0]:
clf_red = svm.SVC(gamma='auto_deprecated')
clf_red.fit(Redundancy_Score_Vec_2_norm, Redundancy_Score_Y)  

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
predict_red_3 = clf_red.predict(Redundancy_test_Vec_2_norm)
red_MSE_3 = mean_squared_error(Redundancy_test_Y,predict_red_3)
red_pearsonr_3 = pearsonr(Redundancy_test_Y,predict_red_3)
print('The MSE is '+str(red_MSE_3) +'. The pearson is '+str(red_pearsonr_3) +'.')

The MSE is 0.8031088082901554. The pearson is (0.12965330171063671, 0.07232078748755091).


#### 4.3.1

In [0]:
def count_noun_chunks(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    chunk_dic = {}
    for chunk in doc.noun_chunks:
        if str(chunk) in chunk_dic:
            chunk_dic[str(chunk)] +=1
        if str(chunk) not in chunk_dic:
            chunk_dic[str(chunk)] =1
    repeated_chunks = 0
    for i in chunk_dic: 
        if chunk_dic[i]>1:
            repeated_chunks+=1
            
    return repeated_chunks

In [0]:
Coherence_Score_Vec = []
Coherence_Score_Y = []
for i in Summary_list:
    temp_text = i[1].lower().replace('\n',' ')
    temp_vec = [count_noun_chunks(temp_text)]
    Coherence_Score_Vec.append(temp_vec)
    Coherence_Score_Y.append(summary_vector(train_data, i[0])[0])
    

In [0]:
Coherence_Score_Vec_norm = normalization(Coherence_Score_Vec)

In [0]:
Coherence_test_Vec = []
Coherence_test_Y = []
for i in test_summary:
    temp_text = i[1].lower().replace('\n',' ')
    temp_vec = [count_noun_chunks(temp_text)]
    Coherence_test_Vec.append(temp_vec)
    Coherence_test_Y.append(summary_vector(test_data, i[0])[0])

In [0]:
Coherence_test_Vec_norm = normalization(Coherence_test_Vec)

In [0]:
clf_coh = svm.SVC(gamma='auto_deprecated')
clf_coh.fit(Coherence_Score_Vec_norm, Coherence_Score_Y)  

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
predict_coh = clf_coh.predict(Coherence_test_Vec_norm)

In [0]:
coh_MSE_1 = mean_squared_error(Coherence_test_Y,predict_coh)

In [0]:
coh_pearsonr_1 = pearsonr(Coherence_test_Y,predict_coh)

/Users/xizheng/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3038: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [0]:
print('The MSE is '+str(coh_MSE_1) +'. The pearson is '+str(coh_pearsonr_1) +'.')

The MSE is 1.2124352331606219. The pearson is (nan, 1.0).
